In [1]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import pytesseract
from autocorrect import Speller

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' 

def adjust_dpi(image, dpi=300):
    pil_image = Image.fromarray(image)
    pil_image.save("temp_image.png", dpi=(dpi, dpi))
    return cv2.imread("temp_image.png")

def adjust_text_size(image, target_height=32):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        
        if h < target_height:
            scale_factor = target_height / h
            image = cv2.resize(image, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)
            break
    cv2.imwrite('adjust_text_image.jpg',image)
    return image

def deskew(image):

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    coords = np.column_stack(np.where(gray > 0))
    angle = cv2.minAreaRect(coords)[-1]
    
    # # Correct the angle
    # if angle < -45:
    #     angle = -(90 + angle)
    # else:
    #     angle = -angle
    
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    deskewed = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    cv2.imwrite('deskew.jpg',deskewed)
    
    return deskewed

def fix_illumination(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    
    l, a, b = cv2.split(lab)
    
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    
    limg = cv2.merge((cl, a, b))
    
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    cv2.imwrite('illumination.jpg',final)
    
    return final

def binarize_and_denoise(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    blurred = cv2.medianBlur(gray, 3)
    
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    cv2.imwrite('binary.jpg',binary)
    
    return binary

def autocorrect(text):
    spell = Speller(lang='en')
    return spell(text)


image_path = 'E:/ML/OCR/sign2.jpg'
image = cv2.imread(image_path)

image = adjust_dpi(image, dpi=300)
image = adjust_text_size(image, target_height=32)
# image = deskew(image)
# image = fix_illumination(image)
preprocessed_image = binarize_and_denoise(image)

cv2.imwrite('preprocessed_image.png', preprocessed_image)

text = pytesseract.image_to_string(preprocessed_image, lang='eng')
corrected_text=autocorrect(text)

print(text)
print(corrected_text)


Fire Evacuation

tf you dlecovered « fire:

© Manually activate the fire slarn system

© Determine if It is aate for you to attempt to extinguish the fire

© immediately exit the building, using the stairs and closing doors behind you. Don't use the lift!

« Call 4403 399% to provide more detells about the fire

Once Alarm le activated:

© Weik to the neerest exit closing doors between you and the fire

© Asslet people who have special needs

© Call the Fire Team for assistance 4403 3999

¢  Notily the fire personnel M you suspect someone is trapped inside the buliding

© Gather st designated buliding assembly area

¢ Do not attempt to re-enter the building unises instructed to do so by the Civit
Gefene and QU Fire Teen

Suspected Gas Leak
Alert area occupants and heve them evacuate the area
Cail Fire Team 4403 3999 or Health & Safety 4403 3609

‘Tum off ignition sources of flammabie gas, if It une safe to do so

M possible, ventiiate the affected ares (Only if It can be done safely an